### Tanh

In [1]:
import numpy as np

def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

### Relu

In [2]:
def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)

### Abstract class Layer

In [3]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

### Fully-connected layer

In [4]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### Activation Layer

In [5]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

### Dropout Layer

In [6]:
class DropoutLayer(Layer):
    def __init__(self, p):
        self.p = p

    def forward_propagation(self, input_data):
        self.mask = np.random.binomial(1, 1 - self.p, size=input_data.shape)
        self.output = input_data * self.mask
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.mask

### Network class

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

### Solve the XOR problem without Dropout Layer

In [20]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(10, 1))
net.add(ActivationLayer(tanh, tanh_prime))
#net.add(DropoutLayer(p=0.1))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)
out = net.predict(x_train)
print(out)

epoch 1/500   error=0.698956
epoch 2/500   error=0.474182
epoch 3/500   error=0.348177
epoch 4/500   error=0.323970
epoch 5/500   error=0.317388
epoch 6/500   error=0.314884
epoch 7/500   error=0.313556
epoch 8/500   error=0.312545
epoch 9/500   error=0.311595
epoch 10/500   error=0.310635
epoch 11/500   error=0.309654
epoch 12/500   error=0.308653
epoch 13/500   error=0.307637
epoch 14/500   error=0.306609
epoch 15/500   error=0.305570
epoch 16/500   error=0.304522
epoch 17/500   error=0.303462
epoch 18/500   error=0.302391
epoch 19/500   error=0.301304
epoch 20/500   error=0.300200
epoch 21/500   error=0.299075
epoch 22/500   error=0.297927
epoch 23/500   error=0.296751
epoch 24/500   error=0.295544
epoch 25/500   error=0.294302
epoch 26/500   error=0.293021
epoch 27/500   error=0.291699
epoch 28/500   error=0.290329
epoch 29/500   error=0.288910
epoch 30/500   error=0.287437
epoch 31/500   error=0.285906
epoch 32/500   error=0.284314
epoch 33/500   error=0.282658
epoch 34/500   erro

### Solve the XOR problem with Dropout Layer

In [19]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(10, 1))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(DropoutLayer(p=0.1))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)
out = net.predict(x_train)
print(out)

epoch 1/500   error=0.698956
epoch 2/500   error=0.474182
epoch 3/500   error=0.348177
epoch 4/500   error=0.411303
epoch 5/500   error=0.311721
epoch 6/500   error=0.218378
epoch 7/500   error=0.310327
epoch 8/500   error=0.311594
epoch 9/500   error=0.311889
epoch 10/500   error=0.311422
epoch 11/500   error=0.310600
epoch 12/500   error=0.309632
epoch 13/500   error=0.308604
epoch 14/500   error=0.084907
epoch 15/500   error=0.306404
epoch 16/500   error=0.302180
epoch 17/500   error=0.302471
epoch 18/500   error=0.302800
epoch 19/500   error=0.302429
epoch 20/500   error=0.301619
epoch 21/500   error=0.300601
epoch 22/500   error=0.299484
epoch 23/500   error=0.298312
epoch 24/500   error=0.620599
epoch 25/500   error=0.430504
epoch 26/500   error=0.366088
epoch 27/500   error=0.307657
epoch 28/500   error=0.298566
epoch 29/500   error=0.294816
epoch 30/500   error=0.292409
epoch 31/500   error=0.290471
epoch 32/500   error=0.288711
epoch 33/500   error=0.287010
epoch 34/500   erro

### Solve the MNIST problem with TANH

In [21]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])

epoch 1/20   error=0.042247
epoch 2/20   error=0.021373
epoch 3/20   error=0.017366
epoch 4/20   error=0.015063
epoch 5/20   error=0.013448
epoch 6/20   error=0.012274
epoch 7/20   error=0.011323
epoch 8/20   error=0.010582
epoch 9/20   error=0.009878
epoch 10/20   error=0.009291
epoch 11/20   error=0.008769
epoch 12/20   error=0.008260
epoch 13/20   error=0.007880
epoch 14/20   error=0.007516
epoch 15/20   error=0.007213
epoch 16/20   error=0.006890
epoch 17/20   error=0.006653
epoch 18/20   error=0.006448
epoch 19/20   error=0.006197
epoch 20/20   error=0.006017


predicted values : 
[array([[ 4.54926533e-03, -6.59751279e-04, -1.80151899e-02,
         2.71950495e-03,  4.32174363e-02, -6.55587823e-03,
         7.91355701e-04,  9.89372049e-01, -1.38102060e-02,
         1.13200466e-02]]), array([[0.0262188 , 0.00856112, 0.96242119, 0.03664727, 0.0156369 ,
        0.04104411, 0.00994825, 0.01766055, 0.06400523, 0.02315558]]), array([[ 0.00277871,  0.993914  , -0.00762733,  0.0013321 , -0

### Solve the MNIST problem with RELU

In [10]:
from keras.datasets import mnist
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))
net.add(FCLayer(100, 50))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))
net.add(FCLayer(50, 10))
net.add(ActivationLayer(tanh, tanh_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])

epoch 1/20   error=0.083660
epoch 2/20   error=0.074360
epoch 3/20   error=0.073893
epoch 4/20   error=0.073555
epoch 5/20   error=0.073185
epoch 6/20   error=0.068493
epoch 7/20   error=0.066521
epoch 8/20   error=0.065671
epoch 9/20   error=0.065261
epoch 10/20   error=0.065193
epoch 11/20   error=0.062004
epoch 12/20   error=0.057549
epoch 13/20   error=0.054199
epoch 14/20   error=0.048081
epoch 15/20   error=0.042809
epoch 16/20   error=0.041666
epoch 17/20   error=0.035522
epoch 18/20   error=0.033087
epoch 19/20   error=0.031982
epoch 20/20   error=0.026341


predicted values : 
[array([[0.04130301, 0.00277958, 0.01408205, 0.02674866, 0.28836682,
        0.0261455 , 0.00971734, 0.04419697, 0.04137309, 0.31029831]]), array([[0.04130301, 0.00277958, 0.01408205, 0.02674866, 0.28836682,
        0.0261455 , 0.00971734, 0.04419697, 0.04137309, 0.31029831]]), array([[0.00951132, 0.98396637, 0.00489434, 0.00146406, 0.11323585,
        0.00496971, 0.00325671, 0.01527928, 0.00674477, 0.12

### Comparativa de Resultados

##### Solve the XOR problem without Dropout Layer vs Solve the XOR problem with Dropout Layer

| Sample | Without Dropout | With Dropout |
|--------|-----------------|--------------|
| 1      | 0.698956        | 0.698956     |
| 2      | 0.474182        | 0.474182     |
| 3      | 0.348177        | 0.348177     |
| ...    | ...             | ...          |
| 500    | 0.000646        | 0.000770     |

##### Solve the XOR problem without Dropout Layer vs Solve the XOR problem with Dropout Layer

| Epoch | Error with TANH | Error with RELU |
|-------|------------------|-----------------|
| 1     | 0.042247         | 0.083660        |
| 2     | 0.021373         | 0.074360        |
| 3     | 0.017366         | 0.073893        |
| ...   | ...              | ...             |
| 20    | 0.006017         | 0.026341        |


### Explicación de la Elección de ReLU y tanh

##### ReLU (Rectified Linear Unit):
Es una función de activación no lineal que se define como f(x) = max(0, x).
Es popular en redes neuronales profundas porque reduce el problema del gradiente desvaneciente y permite un entrenamiento más rápido.
En las capas ocultas, ayuda a aprender características más complejas y a mejorar la capacidad de representación de la red.

##### tanh (Tangente Hiperbólica):
Es una función de activación no lineal que se define como f(x) = (e^x - e^-x) / (e^x + e^-x).
La salida de tanh está en el rango [-1, 1], lo cual puede ser útil para normalizar las salidas.
En la capa de salida, tanh puede ser beneficiosa para asegurar que las predicciones estén en un rango limitado, lo cual puede ser útil en ciertas aplicaciones de clasificación.

### Conclusiones sobre el Uso de Dropout, TANH y ReLU

##### Dropout
1. **Efectividad del Dropout**:
   - En el problema XOR, el uso del Dropout no mostró una mejora significativa en la reducción del error final. De hecho, el error final fue ligeramente mayor con Dropout (0.000770) comparado con sin Dropout (0.000646).
   - Esto puede deberse a que el problema XOR es relativamente simple y no se beneficia tanto de la regularización que ofrece Dropout.
   
2. **Regularización y Sobreajuste**:
   - Dropout es más efectivo en problemas complejos o cuando se tiene una gran cantidad de datos y modelos profundos, ya que ayuda a prevenir el sobreajuste al desactivar aleatoriamente neuronas durante el entrenamiento.
   - En el caso del problema XOR, la simplicidad del modelo y los datos no requieren tanta regularización.

##### Función de Activación TANH
1. **Convergencia Inicial Rápida**:
   - En el problema MNIST, la función de activación TANH mostró una convergencia inicial más rápida en comparación con ReLU. El error inicial fue menor con TANH (0.042247) que con ReLU (0.083660).
   - Esto sugiere que TANH puede ser beneficioso para iniciar el aprendizaje, ya que su salida está centrada en cero, lo que puede ayudar a una mejor propagación de gradientes al inicio del entrenamiento.

2. **Error Final Más Bajo**:
   - Después de 20 épocas, el error final con TANH (0.006017) fue significativamente menor que con ReLU (0.026341). Esto indica que TANH puede ofrecer un mejor ajuste final para este conjunto de datos específico.
   - Sin embargo, TANH puede sufrir del problema de gradientes desvanecientes en redes muy profundas, lo que no fue un problema en esta configuración relativamente superficial.

##### Función de Activación ReLU
1. **Capacidad de Aprendizaje en Redes Profundas**:
   - ReLU es conocida por ser más efectiva en redes neuronales profundas, ya que mitiga el problema de gradientes desvanecientes al permitir que solo las neuronas con valores positivos pasen gradientes significativos.
   - Aunque el error inicial y final fueron mayores con ReLU en este caso, en configuraciones más complejas y profundas, ReLU podría mostrar una mejor performance.

2. **Robustez y Eficiencia**:
   - ReLU es computacionalmente eficiente y simple, ya que simplemente pasa valores positivos y bloquea los negativos.
   - Aunque no tuvo el menor error en este experimento, sigue siendo una opción robusta para muchos problemas de aprendizaje profundo.

##### Conclusión General
- **Dropout**: Efectivo en modelos complejos para prevenir sobreajuste, pero no siempre necesario para problemas simples.
- **TANH**: Puede ofrecer una mejor convergencia inicial y ajuste final en problemas menos complejos, pero puede sufrir de gradientes desvanecientes en redes profundas.
- **ReLU**: Eficiente y robusto, especialmente útil en redes profundas y configuraciones más complejas, aunque puede requerir ajustes de hiperparámetros para obtener mejores resultados.